<a href="https://colab.research.google.com/github/NadaSOsman/MobilityPred-COVID-Restrictions/blob/master/Autoregressive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
from os import listdir
from os.path import isfile, join
import gzip
import shutil

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('/root/data')
try:
	os.makedirs(local_download_path)
except: pass

file_list = drive.ListFile(
	{'q': "'1aLmDCh-VHHvZExBxHyesB4W2j9KAnRlt' in parents"}).GetList()

for f in file_list:
	# 3. Create & download by id.
	print('title: %s, id: %s' % (f['title'], f['id']))
	fname = os.path.join(local_download_path, f['title'])
	print('downloading to {}'.format(fname))
	f_ = drive.CreateFile({'id': f['id']})
	f_.GetContentFile(fname)
 

import numpy as np
import tensorflow as tf
import random as rn
import sys

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(1442)
rn.seed(12435)

from tensorflow.compat.v1.keras import backend as K

tf.random.set_seed(1234)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, LSTM, Convolution2D, Flatten, Concatenate, Reshape
from keras.callbacks import LearningRateScheduler
import csv
import random
import gc
from keras import optimizers

INPUT_WIN_SIZE = 10
OUTPUT_WIN_SIZE = 1
MOBILITY_CATEGORIES = 6
TRAINIG_DATA_SAMPLES = 99
TESTING_DATA_SAMPLES = 20
SAMPLE_SIZE = 164
BATCH_SIZE = 90
CONTROL_MEASURES = 8


with open("/root/data/training-moblity.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  training_data = np.array(list(reader)).astype(int)
f.close()

with open("/root/data/testing-mobility.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  testing_data = np.array(list(reader)).astype(int)
f.close()

def build_model():
  inputs = []
  outputs = []
  for i in range(0, MOBILITY_CATEGORIES):
    input = Input(shape=(INPUT_WIN_SIZE,), dtype='int32')
    fc1 = Dense(32, activation='relu')(input)
    d1 = Dropout(0.2)(fc1)
    fc2 = Dense(16, activation='relu')(d1)
    d2 = Dropout(0.2)(fc2)
    fc3 = Dense(8, activation='relu')(d2)
    d3 = Dropout(0.2)(fc3)
    output = Dense(1, activation='linear')(d3)
    
    inputs.append(input)
    outputs.append(output)

  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.001))

  print(model.summary())
  return model

def step_decay(epoch, lr):
	lrate = lr
	if(epoch%1000 == 0 and epoch != 0):
		#lrate = lr/(2.0-min((epoch/100.0 - 1)*0.1, 1.0))
		lrate = lr*0.7
	print(lrate)
	return lrate

global current_training_indeces
def generator(training):
  global current_training_indeces
  if (training == 1):
    start_batch = random.randint(0,TRAINIG_DATA_SAMPLES)
    indeces = [i*SAMPLE_SIZE for i in range(start_batch, min(start_batch+BATCH_SIZE, TRAINIG_DATA_SAMPLES))]
    diff = BATCH_SIZE-len(indeces)
    for i in range (0, diff):
      indeces.append(i*SAMPLE_SIZE)
    current_training_indeces = indeces[:]
    data = np.transpose(training_data)
  elif (training == 2):
    indeces = []
    for i in range(0, TRAINIG_DATA_SAMPLES):
      if (i*SAMPLE_SIZE not in current_training_indeces):
        indeces.append(i*SAMPLE_SIZE)
    data = np.transpose(training_data)
  else:
    indeces = [i*SAMPLE_SIZE for i in range(0,TESTING_DATA_SAMPLES)]
    data = np.transpose(testing_data)
    print(data.shape)

  time_series_input = [[] for i in range(0, MOBILITY_CATEGORIES)]
  time_series_output = [[] for i in range(0, MOBILITY_CATEGORIES)]

  for index in indeces:
    i = index
    while i < index+SAMPLE_SIZE-(INPUT_WIN_SIZE+OUTPUT_WIN_SIZE):
      for c in range(0, MOBILITY_CATEGORIES):
        if (len(data[c][i:i+INPUT_WIN_SIZE]) == 0):
          print(data[c], data[c][i:i+INPUT_WIN_SIZE], i)
        time_series_input[c].append(np.reshape(data[c][i:i+INPUT_WIN_SIZE],(INPUT_WIN_SIZE,)))
        time_series_output[c].append(np.reshape(data[c][i+INPUT_WIN_SIZE:i+INPUT_WIN_SIZE+OUTPUT_WIN_SIZE],(OUTPUT_WIN_SIZE,)))
      i += 1

  for c in range(0, MOBILITY_CATEGORIES):
     time_series_input[c] = np.asarray(time_series_input[c])
     time_series_output[c] = np.asarray(time_series_output[c])

  return time_series_input, time_series_output

def validation_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(2)
		yield time_series_input, time_series_output

def training_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(1)
		yield time_series_input, time_series_output

def train_model(model):
  lrate = LearningRateScheduler(step_decay)
  model.fit_generator(generator = training_batch_generator(), steps_per_epoch=1, epochs=2000, verbose=1, validation_data=validation_batch_generator(), validation_steps=1, callbacks=[lrate])
  return model


model = build_model()
model = train_model(model)

time_series_input, time_series_output = generator(3)
pred = (np.asarray(model.predict(time_series_input)))
true = (np.asarray(time_series_output))

pred_file = open("/root/data/pred.csv", 'w')
true_file = open("/root/data/true.csv", 'w')

for i in range(pred.shape[1]):
  p = ""
  t = ""
  for j in range(pred.shape[0]):
    p += str(pred[j][i][0])+","
    t += str(true[j][i][0])+","
  pred_file.write(p[:-1]+"\n")
  true_file.write(t[:-1]+"\n")
pred_file.close()
true_file.close()

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
	os.makedirs(local_download_path)
except: pass

uploaded = drive.CreateFile({'title': 'pred.csv'})
uploaded.SetContentFile('/root/data/pred.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'true.csv'})
uploaded.SetContentFile('/root/data/true.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))


Streaming output truncated to the last 5000 lines.
Epoch 335/2000
1/1 [==============================] - 1s 763ms/step - loss: 1296.5741 - dense_195_loss: 180.0646 - dense_199_loss: 168.4986 - dense_203_loss: 511.7414 - dense_207_loss: 193.3710 - dense_211_loss: 206.5496 - dense_215_loss: 36.3489 - val_loss: 771.3623 - val_dense_195_loss: 182.3605 - val_dense_199_loss: 158.6805 - val_dense_203_loss: 127.8057 - val_dense_207_loss: 194.1381 - val_dense_211_loss: 87.1254 - val_dense_215_loss: 21.2521
0.0010000000474974513
Epoch 336/2000
1/1 [==============================] - 1s 758ms/step - loss: 1420.2633 - dense_195_loss: 175.3991 - dense_199_loss: 164.7103 - dense_203_loss: 623.9223 - dense_207_loss: 199.3988 - dense_211_loss: 217.8353 - dense_215_loss: 38.9976 - val_loss: 834.3897 - val_dense_195_loss: 166.0345 - val_dense_199_loss: 134.3486 - val_dense_203_loss: 211.0014 - val_dense_207_loss: 181.4225 - val_dense_211_loss: 114.7060 - val_dense_215_loss: 26.8766
0.0010000000474974513
